In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, ranksums
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.multitest import multipletests

In [3]:
def calculate_group_differences(expression_matrix, group_info, group1, group2, output_file, method='t-test'):
    """
    计算两组间的差异，并输出结果到CSV文件。

    参数:
    - expression_matrix: 表达矩阵表格 (DataFrame)，行为基因/特征，列为样本。
    - group_info: 分组信息表格 (DataFrame)，至少包含两列：样本名和分组信息。
    - output_file: 输出CSV文件的路径。
    - method: 差异分析方法，可选 't-test', 'wilcox', 或 'logreg'。

    输出:
    - CSV文件包含以下列：分组信息的均值, FC, log2FC, p-value, adjust p-value。
    """
    # 检查分组信息是否只有两组
    unique_groups = group_info['Group'].unique()
    if len(unique_groups) != 2:
        print("分组信息不是两组，跳过计算。")
        return

    # 确保分组信息中的样本名与表达矩阵中的列名一致
    group_info = group_info.set_index('Sample')
    expression_matrix = expression_matrix[group_info.index]
    expression_matrix.replace(0, np.nan, inplace=True)
    expression_matrix.dropna(thresh=3, inplace=True)


    # 验证输入数据
    if not all(col in group_info.columns for col in ['Sample_ID', 'Group']):
        raise ValueError("group_info必须包含'Sample_ID'和'Group'列")
    
    # 获取各组样本ID
    group1_samples = group_info[group_info['Group'] == group1]['Sample_ID'].tolist()
    group2_samples = group_info[group_info['Group'] == group2]['Sample_ID'].tolist()
    
    # 确保样本存在于表达矩阵中
    common_samples = set(expression_matrix.columns) & set(group1_samples + group2_samples)
    group1_samples = [s for s in group1_samples if s in common_samples]
    group2_samples = [s for s in group2_samples if s in common_samples]
    
    if not group1_samples or not group2_samples:
        raise ValueError("至少一组没有可用的样本数据")

    # 分组
    # group1, group2 = unique_groups
    group1_samples = group_info[group_info['Group'] == group1].index
    group2_samples = group_info[group_info['Group'] == group2].index

    # 计算均值
    group1_mean = expression_matrix[group1_samples].mean(axis=1)
    group2_mean = expression_matrix[group2_samples].mean(axis=1)

    # 计算 count
    group1_count = expression_matrix[group1_samples].count(axis=1)
    group2_count = expression_matrix[group2_samples].count(axis=1)
    total_count = expression_matrix.shape[1]
    
    # 替换空值
    expression_matrix.replace(np.nan, 0, inplace=True)

    # 计算FC (Tumor/NAT)
    fc = group2_mean / group1_mean  # 假设group2是Tumor，group1是NAT
    log2fc = np.log2(fc)

    # 计算p-value
    p_values = []
    if method == 't-test':
        for gene in expression_matrix.index:
            t_stat, p_value = ttest_ind(expression_matrix.loc[gene, group2_samples], expression_matrix.loc[gene, group1_samples])
            p_values.append(p_value)
    elif method == 'wilcox':
        for gene in expression_matrix.index:
            stat, p_value = ranksums(expression_matrix.loc[gene, group2_samples], expression_matrix.loc[gene, group1_samples])
            p_values.append(p_value)
    elif method == 'logreg':
        for gene in expression_matrix.index:
            X = expression_matrix.loc[gene].values.reshape(-1, 1)
            y = (group_info['Group'] == group2).astype(int)
            model = LogisticRegression()
            model.fit(X, y)
            p_value = 2 * (1 - model.score(X, y))  # 近似p值
            p_values.append(p_value)
    else:
        raise ValueError("未知的差异分析方法。请选择 't-test', 'wilcox', 或 'logreg'。")

    p_values = np.array(p_values)

    # 计算adjust p-value (FDR校正)
    adjust_p_values = multipletests(p_values, method='fdr_bh')[1]

    # 构建结果DataFrame
    result_df = pd.DataFrame({
        'Gene': expression_matrix.index,
        f'{group1}_Mean': group1_mean,
        f'{group2}_Mean': group2_mean,
        f'{group1}_Count': group1_count,
        f'{group2}_Count': group2_count,
        'Total_count': total_count,
        'FC': fc,
        'log2FC': log2fc,
        'p-value': p_values,
        'adjust_p-value': adjust_p_values
    })

    # 保存结果到CSV文件
    result_df.to_csv(output_file, index=False)
    print(f"结果已保存到 {output_file}")

    return result_df

In [4]:
def calculate_group_differences(expression_matrix, group_info, group1, group2, output_file, method='t-test'):
    """
    计算两组间的差异，并输出结果到CSV文件。

    参数:
    - expression_matrix: 表达矩阵表格 (DataFrame)，行为基因/特征，列为样本。
    - group_info: 分组信息表格 (DataFrame)，至少包含两列：样本名和分组信息。
    - output_file: 输出CSV文件的路径。
    - method: 差异分析方法，可选 't-test', 'wilcox', 或 'logreg'。

    输出:
    - CSV文件包含以下列：分组信息的均值, FC, log2FC, p-value, adjust p-value。
    """
    from scipy.stats import ttest_ind, ranksums
    from statsmodels.stats.multitest import multipletests
    import numpy as np
    import pandas as pd
    
    # 检查分组信息是否只有两组
    unique_groups = group_info['Group'].unique()
    if len(unique_groups) != 2:
        print("分组信息不是两组，跳过计算。")
        return None

    # 确保分组信息中的样本名与表达矩阵中的列名一致
    group_info = group_info.set_index('Sample_ID')
    common_samples = list(set(expression_matrix.columns) & set(group_info.index))
    expression_matrix = expression_matrix[common_samples]
    group_info = group_info.loc[common_samples]

    # 替换0为NaN并过滤低表达基因
    expression_matrix = expression_matrix.replace(0, np.nan)
    expression_matrix = expression_matrix.dropna(thresh=3)  # 至少3个样本有表达

    # 获取各组样本
    group1_samples = group_info[group_info['Group'] == group1].index
    group2_samples = group_info[group_info['Group'] == group2].index

    # 检查样本数量
    if len(group1_samples) == 0 or len(group2_samples) == 0:
        raise ValueError(f"One or both groups are empty: {group1}={len(group1_samples)}, {group2}={len(group2_samples)}")

    # 计算均值
    group1_mean = expression_matrix[group1_samples].mean(axis=1)
    group2_mean = expression_matrix[group2_samples].mean(axis=1)

    # 计算count
    group1_count = expression_matrix[group1_samples].count(axis=1)
    group2_count = expression_matrix[group2_samples].count(axis=1)
    total_count = expression_matrix.shape[1]

    # 计算FC (group2/group1)
    fc = group2_mean / group1_mean
    log2fc = np.log2(fc)

    # 计算p-value
    p_values = []
    expression_matrix = expression_matrix.fillna(0)  # 填充剩余NaN为0
    
    if method == 't-test':
        for gene in expression_matrix.index:
            try:
                t_stat, p_value = ttest_ind(expression_matrix.loc[gene, group2_samples],
                                           expression_matrix.loc[gene, group1_samples],
                                           equal_var=False)  # Welch's t-test
                p_values.append(p_value)
            except:
                p_values.append(np.nan)
                
    elif method == 'wilcox':
        for gene in expression_matrix.index:
            try:
                # 过滤掉全0的情况
                g1_vals = expression_matrix.loc[gene, group1_samples]
                g2_vals = expression_matrix.loc[gene, group2_samples]
                if (g1_vals == 0).all() and (g2_vals == 0).all():
                    p_values.append(1.0)  # 无差异
                else:
                    stat, p_value = ranksums(g2_vals, g1_vals)
                    p_values.append(p_value)
            except:
                p_values.append(np.nan)
                
    elif method == 'logreg':
        from sklearn.linear_model import LogisticRegression
        y = (group_info['Group'] == group2).astype(int)
        for gene in expression_matrix.index:
            try:
                X = expression_matrix.loc[gene].values.reshape(-1, 1)
                model = LogisticRegression(max_iter=1000)
                model.fit(X, y)
                p_value = 2 * (1 - model.score(X, y))  # 近似p值
                p_values.append(p_value)
            except:
                p_values.append(np.nan)
    else:
        raise ValueError("未知的差异分析方法。请选择 't-test', 'wilcox', 或 'logreg'。")

    # 计算adjust p-value (FDR校正)
    # 首先将NaN p-values替换为1（表示无显著性），计算后再恢复NaN
    p_values_arr = np.array(p_values)
    nan_mask = np.isnan(p_values_arr)
    
    # 临时将NaN替换为1用于FDR校正
    temp_p_values = p_values_arr.copy()
    temp_p_values[nan_mask] = 1.0
    
    # 执行FDR校正
    _, adjust_p_values, _, _ = multipletests(temp_p_values, method='fdr_bh')
    
    # 将原本是NaN的位置恢复为NaN
    adjust_p_values[nan_mask] = np.nan

    # 构建结果DataFrame
    result_df = pd.DataFrame({
        'Gene': expression_matrix.index,
        f'{group1}_Mean': group1_mean,
        f'{group2}_Mean': group2_mean,
        f'{group1}_Count': group1_count,
        f'{group2}_Count': group2_count,
        'Total_count': total_count,
        'FC': fc,
        'log2FC': log2fc,
        'p-value': p_values,
        'adjust_p-value': adjust_p_values
    })

    # 保存结果到CSV文件
    result_df.to_csv(output_file, index=False)
    print(f"结果已保存到 {output_file}")

    return result_df

In [83]:
expression_matrix = pd.read_csv('/Volumes/Extreme_Pro/Samsung_T5/东方肝胆外科合作项目/data/RNA-seq/Chemo_Immuno/DEP/R_Chemo_R_Immuno/R_Chemo_R_Immuno_matrix.csv', index_col=0)
expression_matrix

T_238402     T_238867      T_239051      T_240194     T_240610  \
SYMBOL                                                                       
ATAD3B      4.647684     3.884499      4.584523     10.410701     4.308672   
PRDM16      1.363787     2.076841      0.052503      0.114938     0.358566   
PEX10       3.567714     3.486233      5.249250      5.145016     2.095672   
PEX14       6.578514     4.107752     14.041770     12.358503     2.996589   
PLCH2       0.136035     0.064509      1.219934      5.475634     1.511998   
...              ...          ...           ...           ...          ...   
MT-ND6  14208.770790  1946.467753   1227.230211   1034.101473  1586.453518   
MT-TE     663.122100    93.705616     14.863752      8.134804    16.027970   
MT-CYB  52112.633960  6061.630640  11424.056810  11736.716300  4668.027911   
MT-TT      82.562633     5.762645     24.419021     18.899040     0.000000   
MT-TP   17054.618520  2102.094191    876.930125    492.514542  2819.037004   

           T_242062     T_242127     T_242471     T_250450     T_251611  \
SYMBOL                                                                    
ATAD3B     4.998995     2.812769     5.165169     2.765437     3.402105   
PRDM16     0.058180     0.208013     0.479785     0.093400     0.152020   
PEX10      2.542701     4.063374     3.455357     3.473355     3.751421   
PEX14      5.680090     7.764326     7.090810    10.391797     8.608873   
PLCH2      0.109187     0.428934     1.767715     0.177232     1.068394   
...             ...          ...          ...          ...          ...   
MT-ND6   460.176425   312.860656   931.114288  1875.040102  1472.207880   
MT-TE     50.046327     6.039864    53.087608   149.395503    86.074054   
MT-CYB  3549.085668  5017.398247  3242.629640  7946.464667  8471.935537   
MT-TT      0.662293     5.525103     6.500080     1.382179     7.498876   
MT-TP    650.527716  2076.092325  1953.332803  1269.084077  1282.197498   

            T_249623     T_251610     T_252216     T_252234     T_259889  \
SYMBOL                                                                     
ATAD3B      1.638998     3.372115     2.583519     3.715302     6.402791   
PRDM16      0.115936     0.131921     0.074684     0.194914     1.032753   
PEX10       2.058401     3.367540     5.743271     2.624681     2.656979   
PEX14       2.848242     5.172001     6.744844     2.875431     6.437494   
PLCH2       0.000000     1.389179     1.012265     1.044162     3.920124   
...              ...          ...          ...          ...          ...   
MT-ND6  16581.040610   560.697043  2248.985562   534.336824  1455.614041   
MT-TE     301.548487    35.192337    91.620266    13.795077    24.364495   
MT-CYB  27321.428760  3078.456786  8388.149474  3242.506339  4567.542066   
MT-TT     257.351213     3.754285     3.684032    26.097180     0.000000   
MT-TP   16071.394020   733.145532  1168.054753  1667.088687  2491.383627   

           T_241815     T_241832     T_183331     T_237916  
SYMBOL                                                      
ATAD3B     3.424163     2.305052     3.246631    15.654151  
PRDM16     0.418049     0.308420     0.915904     0.712621  
PEX10      2.370445     2.159982     2.461080     3.633793  
PEX14      4.041545     4.440458     4.224509     8.951129  
PLCH2      0.800662     0.604611     0.603116     1.788220  
...             ...          ...          ...          ...  
MT-ND6   363.926722   517.275927   136.452358   705.094359  
MT-TE     10.934525    20.718606     5.248860    12.207439  
MT-CYB  2351.625845  1310.421967  2941.237439  9433.955828  
MT-TT      4.034664     1.547169    14.816100    27.539750  
MT-TP   1284.447253   397.190031   250.856902  2011.903182  

[16850 rows x 19 columns]

In [84]:
outpath = r"/Volumes/Extreme_Pro/Samsung_T5/东方肝胆外科合作项目/data/RNA-seq/Chemo_Immuno/DEP/R_Chemo_R_Immuno/"

In [85]:
cancer_type = pd.read_csv("/Volumes/Extreme_Pro/Samsung_T5/东方肝胆外科合作项目/data/RNA-seq/Chemo_Immuno/DEP/R_Chemo_R_Immuno/R_Chemo_R_Immuno_ana.csv")
cancer_type

Sample_ID      Type Response (RECIST)   Response  Gender  Age         OS  \
0   T_238402   R_Chemo                PR  Responder    Male   57  10.000000   
1   T_238867   R_Chemo                PR  Responder  Female   74   4.000000   
2   T_239051   R_Chemo                PR  Responder    Male   54   9.000000   
3   T_240194   R_Chemo                PR  Responder    Male   61   5.000000   
4   T_240610   R_Chemo                PR  Responder  Female   63   6.000000   
5   T_242062   R_Chemo                PR  Responder    Male   38   9.700000   
6   T_242127   R_Chemo                PR  Responder    Male   64   5.666667   
7   T_242471   R_Chemo                PR  Responder  Female   55   9.366667   
8   T_250450   R_Chemo                PR  Responder  Female   34  15.633333   
9   T_251611   R_Chemo                PR  Responder    Male   65  14.166667   
10  T_249623  R_Immuno                PR  Responder    Male   66  16.600000   
11  T_251610  R_Immuno                PR  Responder    Male   51  14.166667   
12  T_252216  R_Immuno                PR  Responder    Male   65   9.800000   
13  T_252234  R_Immuno                PR  Responder    Male   68  13.566667   
14  T_259889  R_Immuno                PR  Responder  Female   67   5.633333   
15  T_241815  R_Immuno                PR  Responder    Male   39  13.933333   
16  T_241832  R_Immuno                PR  Responder  Female   46  10.233333   
17  T_183331  R_Immuno                CR  Responder    Male   59  10.000000   
18  T_237916  R_Immuno                CR  Responder    Male   61  10.000000   

    Status        PFS  progress  ...    HLA-DPB1        IDO1       LAG3  \
0        0  10.000000         0  ...   17.043761    0.322369   1.132123   
1        1   4.000000         1  ...    3.726628    0.181735   0.041485   
2        0   6.000000         1  ...   18.166966    0.168021   3.763534   
3        1   5.000000         1  ...   16.696444    0.268206   7.101982   
4        1   6.000000         1  ...    8.447781    1.109735   0.502622   
5        1   7.666667         1  ...   16.925382    0.288234   0.986938   
6        1   5.666667         1  ...    8.389680    0.119481   1.448945   
7        1   5.266667         1  ...   18.143271    0.491979   0.799094   
8        0  15.633333         0  ...   22.989377    0.392304   2.179103   
9        0  14.166667         0  ...   29.143946    0.402033   1.862450   
10       0  16.600000         0  ...   48.735085    2.394222   3.797951   
11       0   7.300000         1  ...   32.313304    3.907125   3.372959   
12       0   9.800000         0  ...   60.390747    1.487132   2.280821   
13       0  13.566667         0  ...    5.377796    0.339578   0.350073   
14       0   5.633333         0  ...   34.795357    6.790712   4.408630   
15       0  13.933333         0  ...  120.939488  350.762473  51.279580   
16       0  10.233333         0  ...  110.462860    4.581624   2.556170   
17       0  10.000000         0  ...    7.173645   13.802998   6.426440   
18       0  10.000000         0  ...  146.495542    4.071501   3.945612   

       TIGIT      ICOS       CD27      CD274  PDCD1LG2     CTLA4     HAVCR2  
0   0.213005  0.295988   0.392561   0.779037  0.497911  0.646686   2.719774  
1   0.000000  0.024102   0.479494   0.401197  0.026065  0.000000   0.157881  
2   0.408921  0.139272   0.720380   0.820308  0.331344  0.136929   1.778970  
3   0.460763  0.047427   0.896346   0.926090  0.461597  0.174860   1.592179  
4   0.312700  0.280337   0.325328   2.692137  5.608471  0.401948   1.377248  
5   0.358819  0.365646   0.892743   2.553730  1.851262  0.674054   6.559424  
6   0.085526  0.316921   1.812645   0.836951  1.649352  0.194743   5.806224  
7   0.119194  0.163120   1.273087   1.397778  3.500865  0.154208   8.209005  
8   0.269583  0.208115   0.345022   1.243557  3.413391  0.085256   3.010499  
9   0.501074  0.219549   1.372714   0.883509  3.154338  0.308365   5.778243  
10  1.185142  1.560742   0.214135   3.100994  1.978811  1.12440

In [86]:
cancer_types = cancer_type['Type'].unique()
cancer_types

array(['R_Chemo', 'R_Immuno'], dtype=object)

In [79]:
expression_matrix = expression_matrix[cancer_type['Sample_ID'].tolist()]
expression_matrix

T_240147     T_241235      T_242038     T_242444      T_244938  \
SYMBOL                                                                      
ATAD3B     2.527509     2.331942      2.460850     1.924898      2.370334   
PRDM16     0.137172     0.016004      0.010525     0.801938      0.252774   
PEX10      3.684147     2.424659      5.164162     2.291524      5.312348   
PEX14      5.209140     4.531436     12.936899     3.351609      7.200850   
PLCH2      0.303673     0.101226      0.256771     1.154040      0.362300   
...             ...          ...           ...          ...           ...   
MT-ND6   986.952354  1146.059241   4494.936989  1779.171097   3692.972505   
MT-TE     33.979276    46.061740    114.712148    39.414878     96.236889   
MT-CYB  4981.690357  5844.307036  16650.376150  5322.586570  13655.617420   
MT-TT      5.074827     7.104903      1.557485    12.636649      1.289888   
MT-TP   2844.022605  1873.395817   2999.166195  4390.863831   3610.625164   

           T_248561     T_250599     T_252664      T_253154     T_258375  \
SYMBOL                                                                     
ATAD3B     5.075480     2.731868     5.265348      3.864366     2.663702   
PRDM16     0.017142     0.079891     1.938310      0.370307     0.568922   
PEX10      5.378688     3.038519     2.744660      2.839971     2.762170   
PEX14      8.122877     8.854938     5.583458     15.811962     3.015439   
PLCH2      0.499535     0.242820     0.105764      0.699637     0.077683   
...             ...          ...          ...           ...          ...   
MT-ND6  1244.370042   975.812959   993.477961   5095.237738   749.120425   
MT-TE     44.282830    45.234728    36.450020    112.776500    35.075873   
MT-CYB  2999.205395  5678.632495  6377.902600  20969.178900  2493.111964   
MT-TT      9.512812     5.373960    16.628439      0.000000     1.122558   
MT-TP    660.468947   970.157926  1714.807764   2733.545392   531.333152   

           T_257310      T_249623     T_251610     T_252216     T_252234  \
SYMBOL                                                                     
ATAD3B     2.233076      1.638998     3.372115     2.583519     3.715302   
PRDM16     1.259183      0.115936     0.131921     0.074684     0.194914   
PEX10      1.957493      2.058401     3.367540     5.743271     2.624681   
PEX14      4.390371      2.848242     5.172001     6.744844     2.875431   
PLCH2      0.346650      0.000000     1.389179     1.012265     1.044162   
...             ...           ...          ...          ...          ...   
MT-ND6  2209.546327  16581.040610   560.697043  2248.985562   534.336824   
MT-TE     72.696686    301.548487    35.192337    91.620266    13.795077   
MT-CYB  9698.640105  27321.428760  3078.456786  8388.149474  3242.506339   
MT-TT      4.120541    257.351213     3.754285     3.684032    26.097180   
MT-TP   2360.948529  16071.394020   733.145532  1168.054753  1667.088687   

           T_259889     T_241815     T_241832     T_183331     T_237916  
SYMBOL                                                                   
ATAD3B     6.402791     3.424163     2.305052     3.246631    15.654151  
PRDM16     1.032753     0.418049     0.308420     0.915904     0.712621  
PEX10      2.656979     2.370445     2.159982     2.461080     3.633793  
PEX14      6.437494     4.041545     4.440458     4.224509     8.951129  
PLCH2      3.920124     0.800662     0.604611     0.603116     1.788220  
...             ...          ...          ...          ...          ...  
MT-ND6  1455.614041   363.926722   517.275927   136.452358   705.094359  
MT-TE     24.364495    10.934525    20.718606     5.248860    12.207439  
MT-CYB  4567.542066  2351.625845  1310.421967  2941.237439  9433.955828  
MT-TT      0.000000     4.034664     1.547169    14.816100    27.539750  
MT-TP   2491.383627  1284.447253   397.190031   250.856902  2011.903182  

[16850 rows x 20 columns]

In [45]:
expression_matrix = expression_matrix.replace(np.nan, 0)
expression_matrix

T_240147     T_241235      T_242038     T_242444      T_244938  \
SYMBOL                                                                      
ATAD3B     2.527509     2.331942      2.460850     1.924898      2.370334   
PRDM16     0.137172     0.016004      0.010525     0.801938      0.252774   
PEX10      3.684147     2.424659      5.164162     2.291524      5.312348   
PEX14      5.209140     4.531436     12.936899     3.351609      7.200850   
PLCH2      0.303673     0.101226      0.256771     1.154040      0.362300   
...             ...          ...           ...          ...           ...   
MT-ND6   986.952354  1146.059241   4494.936989  1779.171097   3692.972505   
MT-TE     33.979276    46.061740    114.712148    39.414878     96.236889   
MT-CYB  4981.690357  5844.307036  16650.376150  5322.586570  13655.617420   
MT-TT      5.074827     7.104903      1.557485    12.636649      1.289888   
MT-TP   2844.022605  1873.395817   2999.166195  4390.863831   3610.625164   

           T_248561     T_250599     T_252664      T_253154     T_258375  \
SYMBOL                                                                     
ATAD3B     5.075480     2.731868     5.265348      3.864366     2.663702   
PRDM16     0.017142     0.079891     1.938310      0.370307     0.568922   
PEX10      5.378688     3.038519     2.744660      2.839971     2.762170   
PEX14      8.122877     8.854938     5.583458     15.811962     3.015439   
PLCH2      0.499535     0.242820     0.105764      0.699637     0.077683   
...             ...          ...          ...           ...          ...   
MT-ND6  1244.370042   975.812959   993.477961   5095.237738   749.120425   
MT-TE     44.282830    45.234728    36.450020    112.776500    35.075873   
MT-CYB  2999.205395  5678.632495  6377.902600  20969.178900  2493.111964   
MT-TT      9.512812     5.373960    16.628439      0.000000     1.122558   
MT-TP    660.468947   970.157926  1714.807764   2733.545392   531.333152   

           T_257310      T_249623     T_251610     T_252216     T_252234  \
SYMBOL                                                                     
ATAD3B     2.233076      1.638998     3.372115     2.583519     3.715302   
PRDM16     1.259183      0.115936     0.131921     0.074684     0.194914   
PEX10      1.957493      2.058401     3.367540     5.743271     2.624681   
PEX14      4.390371      2.848242     5.172001     6.744844     2.875431   
PLCH2      0.346650      0.000000     1.389179     1.012265     1.044162   
...             ...           ...          ...          ...          ...   
MT-ND6  2209.546327  16581.040610   560.697043  2248.985562   534.336824   
MT-TE     72.696686    301.548487    35.192337    91.620266    13.795077   
MT-CYB  9698.640105  27321.428760  3078.456786  8388.149474  3242.506339   
MT-TT      4.120541    257.351213     3.754285     3.684032    26.097180   
MT-TP   2360.948529  16071.394020   733.145532  1168.054753  1667.088687   

           T_259889     T_241815     T_241832     T_183331     T_237916  
SYMBOL                                                                   
ATAD3B     6.402791     3.424163     2.305052     3.246631    15.654151  
PRDM16     1.032753     0.418049     0.308420     0.915904     0.712621  
PEX10      2.656979     2.370445     2.159982     2.461080     3.633793  
PEX14      6.437494     4.041545     4.440458     4.224509     8.951129  
PLCH2      3.920124     0.800662     0.604611     0.603116     1.788220  
...             ...          ...          ...          ...          ...  
MT-ND6  1455.614041   363.926722   517.275927   136.452358   705.094359  
MT-TE     24.364495    10.934525    20.718606     5.248860    12.207439  
MT-CYB  4567.542066  2351.625845  1310.421967  2941.237439  9433.955828  
MT-TT      0.000000     4.034664     1.547169    14.816100    27.539750  
MT-TP   2491.383627  1284.447253   397.190031   250.856902  2011.903182  

[16850 rows x 20 columns]

In [87]:
cancer_types = cancer_type['Type'].unique()
df = cancer_type.copy()
df['Sample'] = df['Sample_ID']

In [38]:
# for cancer in cancer_types:
#     df['Group'] = df['Type'].apply(lambda x: cancer if x == cancer else "The Rest")
#     df = df.sort_values("Group")

In [72]:
df

Sample_ID     Type Response (RECIST)       Response  Gender  Age  \
0   T_240147       NR                SD  Non_responder    Male   64   
1   T_241235       NR                SD  Non_responder    Male   51   
2   T_242038       NR                SD  Non_responder    Male   66   
3   T_242444       NR                SD  Non_responder  Female   78   
4   T_244938       NR                SD  Non_responder    Male   64   
5   T_248561       NR                SD  Non_responder  Female   70   
6   T_250599       NR                SD  Non_responder  Female   51   
7   T_252664       NR                SD  Non_responder    Male   51   
8   T_253154       NR                SD  Non_responder  Female   57   
9   T_258375       NR                SD  Non_responder    Male   48   
10  T_257310       NR                PD  Non_responder  Female   58   
11  T_238402  R_Chemo                PR      Responder    Male   57   
12  T_238867  R_Chemo                PR      Responder  Female   74   
13  T_239051  R_Chemo                PR      Responder    Male   54   
14  T_240194  R_Chemo                PR      Responder    Male   61   
15  T_240610  R_Chemo                PR      Responder  Female   63   
16  T_242062  R_Chemo                PR      Responder    Male   38   
17  T_242127  R_Chemo                PR      Responder    Male   64   
18  T_242471  R_Chemo                PR      Responder  Female   55   
19  T_250450  R_Chemo                PR      Responder  Female   34   
20  T_251611  R_Chemo                PR      Responder    Male   65   

           OS  Status        PFS  progress  ...      IDO1      LAG3     TIGIT  \
0    5.000000       1   5.000000         1  ...  0.506806  0.237312  0.632535   
1    3.000000       1   2.000000         1  ...  2.016587  1.125253  0.598782   
2    5.833333       1   2.933333         1  ...  0.117883  4.776423  0.354404   
3    8.333333       1   6.333333         1  ...  0.478222  0.154255  0.190506   
4   12.866667       1   5.300000         1  ...  0.561368  2.702181  0.335443   
5   13.366667       1  12.366667         1  ...  1.896017  1.985986  0.529132   
6   15.433333       0   5.800000         1  ...  0.366070  1.462364  0.279506   
7   13.133333       0  13.133333         0  ...  0.419525  2.259476  0.870870   
8   10.733333       1   4.566667         1  ...  0.251375  4.482972  0.683761   
9    6.933333       0   6.933333         0  ...  0.679714  0.872774  0.401401   
10   3.366667       1   1.900000         1  ...  0.866322  0.929459  0.689575   
11  10.000000       0  10.000000         0  ...  0.322369  1.132123  0.213005   
12   4.000000       1   4.000000         1  ...  0.181735  0.041485  0.000000   
13   9.000000       0   6.000000         1  ...  0.168021  3.763534  0.408921   
14   5.000000       1   5.000000         1  ...  0.268206  7.101982  0.460763   
15   6.000000       1   6.000000         1  ...  1.109735  0.502622  0.312700   
16   9.700000       1   7.666667         1  ...  0.288234  0.986938  0.358819   
17   5.666667       1   5.666667         1  ...  0.119481  1.448945  0.085526   
18   9.366667       1   5.266667         1  ...  0.491979  0.799094  0.119194   
19  15.633333       0  15.633333         0  ...  0.392304  2.179103  0.269583   
20  14.166667       0  14.166667         0  ...  0.402033  1.862450  0.501074   

        ICOS      CD27     CD274  PDCD1LG2     CTLA4     HAVCR2    Sample  
0   1.034069  2.160046  0.728228  0.764142  0.910770   3.005786  T_240147  
1   0.515083  2.010010  1.737600  1.478241  0.730411  10.203160  T_241235  
2   0.039085  1.321859  1.926762  4.057658  0.000000   9.280903  T_242038  
3   0.151665  1.042318  1.765418  1.983061  0.406673   4.707697  T_242444  
4   0.356068  1.094746  1.201970  2.590384  0.238690   7.235744  T_244938  
5   0.907153  3.482754  0.927198  1.514540  0.723685   6.346218  T_248561  
6   0.215776  1.609749  1.467766  2.741781  0.165739   4.218191  T_250599  
7   0.956293  1.003115  2.215169  2.068299  1.474

In [ ]:
cancer_types = cancer_type['Type'].unique()
df = cancer_type.copy()

for cancer in cancer_types:
    df['Group'] = df['Type'].apply(lambda x: cancer if x == cancer else "The Rest")
    # df['Group'] = df['Type']
    # df = df.sort_values("Group")
    if len(df.Group.unique()) == 2:
        dep = calculate_group_differences(expression_matrix, df, group1='The Rest', group2=cancer, output_file=f"{outpath}/DEP/{cancer}_DEP.csv", method='wilcox')
        dep.to_csv(f"{outpath}/{cancer}_the_rest_dep.csv")

        overexpression_dep = dep[(dep['FC'] > 1.5) & (dep['p-value'] < 0.05)]
        overexpression_dep.to_csv(f"{outpath}/DEP/{cancer}_the_rest_overexpression_dep.csv")

结果已保存到 /Volumes/Extreme_Pro/Samsung_T5/东方肝胆外科合作项目/data/RNA-seq/Chemo_Immuno/DEP/R_Chemo_R_Immuno//DEP/R_Chemo_DEP.csv
结果已保存到 /Volumes/Extreme_Pro/Samsung_T5/东方肝胆外科合作项目/data/RNA-seq/Chemo_Immuno/DEP/R_Chemo_R_Immuno//DEP/R_Immuno_DEP.csv


## The GSVA Pataway

In [57]:
metadata = pd.read_csv("/Volumes/Extreme_Pro/Samsung_T5/东方肝胆外科合作项目/data/RNA-seq/Chemo_Immuno/RNA_anatation_v3_boxplot.csv")
metadata

Sample_ID      Type Response (RECIST)       Response  Gender  Age  \
0   T_240147        NR                SD  Non_responder    Male   64   
1   T_241235        NR                SD  Non_responder    Male   51   
2   T_242038        NR                SD  Non_responder    Male   66   
3   T_242444        NR                SD  Non_responder  Female   78   
4   T_244938        NR                SD  Non_responder    Male   64   
5   T_248561        NR                SD  Non_responder  Female   70   
6   T_250599        NR                SD  Non_responder  Female   51   
7   T_252664        NR                SD  Non_responder    Male   51   
8   T_253154        NR                SD  Non_responder  Female   57   
9   T_258375        NR                SD  Non_responder    Male   48   
10  T_257310        NR                PD  Non_responder  Female   58   
11  T_238402   R_Chemo                PR      Responder    Male   57   
12  T_238867   R_Chemo                PR      Responder  Female   74   
13  T_239051   R_Chemo                PR      Responder    Male   54   
14  T_240194   R_Chemo                PR      Responder    Male   61   
15  T_240610   R_Chemo                PR      Responder  Female   63   
16  T_242062   R_Chemo                PR      Responder    Male   38   
17  T_242127   R_Chemo                PR      Responder    Male   64   
18  T_242471   R_Chemo                PR      Responder  Female   55   
19  T_250450   R_Chemo                PR      Responder  Female   34   
20  T_251611   R_Chemo                PR      Responder    Male   65   
21  T_249623  R_Immuno                PR      Responder    Male   66   
22  T_251610  R_Immuno                PR      Responder    Male   51   
23  T_252216  R_Immuno                PR      Responder    Male   65   
24  T_252234  R_Immuno                PR      Responder    Male   68   
25  T_259889  R_Immuno                PR      Responder  Female   67   
26  T_241815  R_Immuno                PR      Responder    Male   39   
27  T_241832  R_Immuno                PR      Responder  Female   46   
28  T_183331  R_Immuno                CR      Responder    Male   59   
29  T_237916  R_Immuno                CR      Responder    Male   61   

           OS  Status        PFS  progress  ...    HLA-DPB1        IDO1  \
0    5.000000       1   5.000000         1  ...   14.834029    0.506806   
1    3.000000       1   2.000000         1  ...   27.218682    2.016587   
2    5.833333       1   2.933333         1  ...   28.188627    0.117883   
3    8.333333       1   6.333333         1  ...   13.047819    0.478222   
4   12.866667       1   5.300000         1  ...   34.023708    0.561368   
5   13.366667       1  12.366667         1  ...   39.355571    1.896017   
6   15.433333       0   5.800000         1  ...   26.652045    0.366070   
7   13.133333       0  13.133333         0  ...   24.484119    0.419525   
8   10.733333       1   4.566667         1  ...   25.185683    0.251375   
9    6.933333       0   6.933333         0  ...   52.362541    0.679714   
10   3.366667       1   1.900000         1  ...   48.826801    0.866322   
11  10.000000       0  10.000000         0  ...   17.043761    0.322369   
12   4.000000       1   4.000000         1  ...    3.726628    0.181735   
13   9.000000       0   6.000000         1  ...   18.166966    0.168021   
14   5.000000       1   5.000000         1  ...   16.696444    0.268206   
15   6.000000       1   6.000000         1  ...    8.447781    1.109735   
16   9.700000       1   7.666667         1  ...   16.925382    0.288234   
17   5.666667       1   5.666667         1  ...    8.389680    0.119481   
18   9.366667       1   5.266667         1  ...   18.143271    0.491979   
19  15.633333       0  15.633333         0  ...   22.989377    0.392304   
20  14.166667       0  14.166667         0  ...   29.143946    0.402033   
21  16.600000       0  16.600000         0  ...   48.735085    2.394222   
22  14.166667       0   7.300000         1  .

In [ ]:
expression_matrix = pd.read_csv('/Volumes/Extreme_Pro/Samsung_T5/东方肝胆外科合作项目/data/RNA-seq/Chemo_Immuno/RNA_matrix_FPKM_tnm_v2_cleaned.csv', index_col=0)
expression_matrix

T_240147     T_241235      T_242038     T_242444      T_244938  \
SYMBOL                                                                      
ATAD3B     2.527509     2.331942      2.460850     1.924898      2.370334   
PRDM16     0.137172     0.016004      0.010525     0.801938      0.252774   
PEX10      3.684147     2.424659      5.164162     2.291524      5.312348   
PEX14      5.209140     4.531436     12.936899     3.351609      7.200850   
PLCH2      0.303673     0.101226      0.256771     1.154040      0.362300   
...             ...          ...           ...          ...           ...   
MT-ND6   986.952354  1146.059241   4494.936989  1779.171097   3692.972505   
MT-TE     33.979276    46.061740    114.712148    39.414878     96.236889   
MT-CYB  4981.690357  5844.307036  16650.376150  5322.586570  13655.617420   
MT-TT      5.074827     7.104903      1.557485    12.636649      1.289888   
MT-TP   2844.022605  1873.395817   2999.166195  4390.863831   3610.625164   

           T_248561     T_250599     T_252664      T_253154     T_258375  ...  \
SYMBOL                                                                    ...   
ATAD3B     5.075480     2.731868     5.265348      3.864366     2.663702  ...   
PRDM16     0.017142     0.079891     1.938310      0.370307     0.568922  ...   
PEX10      5.378688     3.038519     2.744660      2.839971     2.762170  ...   
PEX14      8.122877     8.854938     5.583458     15.811962     3.015439  ...   
PLCH2      0.499535     0.242820     0.105764      0.699637     0.077683  ...   
...             ...          ...          ...           ...          ...  ...   
MT-ND6  1244.370042   975.812959   993.477961   5095.237738   749.120425  ...   
MT-TE     44.282830    45.234728    36.450020    112.776500    35.075873  ...   
MT-CYB  2999.205395  5678.632495  6377.902600  20969.178900  2493.111964  ...   
MT-TT      9.512812     5.373960    16.628439      0.000000     1.122558  ...   
MT-TP    660.468947   970.157926  1714.807764   2733.545392   531.333152  ...   

           T_242471      T_249623     T_250450     T_251610     T_251611  \
SYMBOL                                                                     
ATAD3B     5.165169      1.638998     2.765437     3.372115     3.402105   
PRDM16     0.479785      0.115936     0.093400     0.131921     0.152020   
PEX10      3.455357      2.058401     3.473355     3.367540     3.751421   
PEX14      7.090810      2.848242    10.391797     5.172001     8.608873   
PLCH2      1.767715      0.000000     0.177232     1.389179     1.068394   
...             ...           ...          ...          ...          ...   
MT-ND6   931.114288  16581.040610  1875.040102   560.697043  1472.207880   
MT-TE     53.087608    301.548487   149.395503    35.192337    86.074054   
MT-CYB  3242.629640  27321.428760  7946.464667  3078.456786  8471.935537   
MT-TT      6.500080    257.351213     1.382179     3.754285     7.498876   
MT-TP   1953.332803  16071.394020  1269.084077   733.145532  1282.197498   

           T_252216     T_252234     T_259889     T_183331     T_237916  
SYMBOL                                                                   
ATAD3B     2.583519     3.715302     6.402791     3.246631    15.654151  
PRDM16     0.074684     0.194914     1.032753     0.915904     0.712621  
PEX10      5.743271     2.624681     2.656979     2.461080     3.633793  
PEX14      6.744844     2.875431     6.437494     4.224509     8.951129  
PLCH2      1.012265     1.044162     3.920124     0.603116     1.788220  
...             ...          ...          ...          ...          ...  
MT-ND6  2248.985562   534.336824  1455.614041   136.452358   705.094359  
MT-TE     91.620266    13.795077    24.364495     5.248860    12.207439  
MT-CYB  8388.149474  3242.506339  4567.542066  2941.237439  9433.955828  
MT-TT      3.684032    26.097180     0.000000    14.816100    27.539750  
MT-TP   1168.054753  1667.088687  2491.383627   250.856902  2011.903182  

[16

In [69]:
expression_matrix.replace(0, np.nan, inplace = True)
expression_matrix = expression_matrix.dropna(thresh=15, axis = 0)
expression_matrix.replace(np.nan, 0, inplace = True)
expression_matrix.to_csv("/Volumes/Extreme_Pro/Samsung_T5/东方肝胆外科合作项目/data/RNA-seq/Chemo_Immuno/RNA_matrix_FPKM_tnm_v2_cleaned.csv")

/var/folders/_y/7lxblyf94hx20mb568vxwfm00000gn/T/ipykernel_2082/1638310060.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  expression_matrix.replace(0, np.nan, inplace = True)


In [70]:
expression_matrix

T_240147     T_241235      T_242038     T_242444      T_244938  \
SYMBOL                                                                      
ATAD3B     2.527509     2.331942      2.460850     1.924898      2.370334   
PRDM16     0.137172     0.016004      0.010525     0.801938      0.252774   
PEX10      3.684147     2.424659      5.164162     2.291524      5.312348   
PEX14      5.209140     4.531436     12.936899     3.351609      7.200850   
PLCH2      0.303673     0.101226      0.256771     1.154040      0.362300   
...             ...          ...           ...          ...           ...   
MT-ND6   986.952354  1146.059241   4494.936989  1779.171097   3692.972505   
MT-TE     33.979276    46.061740    114.712148    39.414878     96.236889   
MT-CYB  4981.690357  5844.307036  16650.376150  5322.586570  13655.617420   
MT-TT      5.074827     7.104903      1.557485    12.636649      1.289888   
MT-TP   2844.022605  1873.395817   2999.166195  4390.863831   3610.625164   

           T_248561     T_250599     T_252664      T_253154     T_258375  ...  \
SYMBOL                                                                    ...   
ATAD3B     5.075480     2.731868     5.265348      3.864366     2.663702  ...   
PRDM16     0.017142     0.079891     1.938310      0.370307     0.568922  ...   
PEX10      5.378688     3.038519     2.744660      2.839971     2.762170  ...   
PEX14      8.122877     8.854938     5.583458     15.811962     3.015439  ...   
PLCH2      0.499535     0.242820     0.105764      0.699637     0.077683  ...   
...             ...          ...          ...           ...          ...  ...   
MT-ND6  1244.370042   975.812959   993.477961   5095.237738   749.120425  ...   
MT-TE     44.282830    45.234728    36.450020    112.776500    35.075873  ...   
MT-CYB  2999.205395  5678.632495  6377.902600  20969.178900  2493.111964  ...   
MT-TT      9.512812     5.373960    16.628439      0.000000     1.122558  ...   
MT-TP    660.468947   970.157926  1714.807764   2733.545392   531.333152  ...   

           T_242471      T_249623     T_250450     T_251610     T_251611  \
SYMBOL                                                                     
ATAD3B     5.165169      1.638998     2.765437     3.372115     3.402105   
PRDM16     0.479785      0.115936     0.093400     0.131921     0.152020   
PEX10      3.455357      2.058401     3.473355     3.367540     3.751421   
PEX14      7.090810      2.848242    10.391797     5.172001     8.608873   
PLCH2      1.767715      0.000000     0.177232     1.389179     1.068394   
...             ...           ...          ...          ...          ...   
MT-ND6   931.114288  16581.040610  1875.040102   560.697043  1472.207880   
MT-TE     53.087608    301.548487   149.395503    35.192337    86.074054   
MT-CYB  3242.629640  27321.428760  7946.464667  3078.456786  8471.935537   
MT-TT      6.500080    257.351213     1.382179     3.754285     7.498876   
MT-TP   1953.332803  16071.394020  1269.084077   733.145532  1282.197498   

           T_252216     T_252234     T_259889     T_183331     T_237916  
SYMBOL                                                                   
ATAD3B     2.583519     3.715302     6.402791     3.246631    15.654151  
PRDM16     0.074684     0.194914     1.032753     0.915904     0.712621  
PEX10      5.743271     2.624681     2.656979     2.461080     3.633793  
PEX14      6.744844     2.875431     6.437494     4.224509     8.951129  
PLCH2      1.012265     1.044162     3.920124     0.603116     1.788220  
...             ...          ...          ...          ...          ...  
MT-ND6  2248.985562   534.336824  1455.614041   136.452358   705.094359  
MT-TE     91.620266    13.795077    24.364495     5.248860    12.207439  
MT-CYB  8388.149474  3242.506339  4567.542066  2941.237439  9433.955828  
MT-TT      3.684032    26.097180     0.000000    14.816100    27.539750  
MT-TP   1168.054753  1667.088687  2491.383627   250.856902  2011.903182  

[16

In [71]:
metadata

Sample_ID      Type Response (RECIST)       Response  Gender  Age  \
0   T_240147        NR                SD  Non_responder    Male   64   
1   T_241235        NR                SD  Non_responder    Male   51   
2   T_242038        NR                SD  Non_responder    Male   66   
3   T_242444        NR                SD  Non_responder  Female   78   
4   T_244938        NR                SD  Non_responder    Male   64   
5   T_248561        NR                SD  Non_responder  Female   70   
6   T_250599        NR                SD  Non_responder  Female   51   
7   T_252664        NR                SD  Non_responder    Male   51   
8   T_253154        NR                SD  Non_responder  Female   57   
9   T_258375        NR                SD  Non_responder    Male   48   
10  T_257310        NR                PD  Non_responder  Female   58   
11  T_238402   R_Chemo                PR      Responder    Male   57   
12  T_238867   R_Chemo                PR      Responder  Female   74   
13  T_239051   R_Chemo                PR      Responder    Male   54   
14  T_240194   R_Chemo                PR      Responder    Male   61   
15  T_240610   R_Chemo                PR      Responder  Female   63   
16  T_242062   R_Chemo                PR      Responder    Male   38   
17  T_242127   R_Chemo                PR      Responder    Male   64   
18  T_242471   R_Chemo                PR      Responder  Female   55   
19  T_250450   R_Chemo                PR      Responder  Female   34   
20  T_251611   R_Chemo                PR      Responder    Male   65   
21  T_249623  R_Immuno                PR      Responder    Male   66   
22  T_251610  R_Immuno                PR      Responder    Male   51   
23  T_252216  R_Immuno                PR      Responder    Male   65   
24  T_252234  R_Immuno                PR      Responder    Male   68   
25  T_259889  R_Immuno                PR      Responder  Female   67   
26  T_241815  R_Immuno                PR      Responder    Male   39   
27  T_241832  R_Immuno                PR      Responder  Female   46   
28  T_183331  R_Immuno                CR      Responder    Male   59   
29  T_237916  R_Immuno                CR      Responder    Male   61   

           OS  Status        PFS  progress  ...    HLA-DPB1        IDO1  \
0    5.000000       1   5.000000         1  ...   14.834029    0.506806   
1    3.000000       1   2.000000         1  ...   27.218682    2.016587   
2    5.833333       1   2.933333         1  ...   28.188627    0.117883   
3    8.333333       1   6.333333         1  ...   13.047819    0.478222   
4   12.866667       1   5.300000         1  ...   34.023708    0.561368   
5   13.366667       1  12.366667         1  ...   39.355571    1.896017   
6   15.433333       0   5.800000         1  ...   26.652045    0.366070   
7   13.133333       0  13.133333         0  ...   24.484119    0.419525   
8   10.733333       1   4.566667         1  ...   25.185683    0.251375   
9    6.933333       0   6.933333         0  ...   52.362541    0.679714   
10   3.366667       1   1.900000         1  ...   48.826801    0.866322   
11  10.000000       0  10.000000         0  ...   17.043761    0.322369   
12   4.000000       1   4.000000         1  ...    3.726628    0.181735   
13   9.000000       0   6.000000         1  ...   18.166966    0.168021   
14   5.000000       1   5.000000         1  ...   16.696444    0.268206   
15   6.000000       1   6.000000         1  ...    8.447781    1.109735   
16   9.700000       1   7.666667         1  ...   16.925382    0.288234   
17   5.666667       1   5.666667         1  ...    8.389680    0.119481   
18   9.366667       1   5.266667         1  ...   18.143271    0.491979   
19  15.633333       0  15.633333         0  ...   22.989377    0.392304   
20  14.166667       0  14.166667         0  ...   29.143946    0.402033   
21  16.600000       0  16.600000         0  ...   48.735085    2.394222   
22  14.166667       0   7.300000         1  .

In [91]:
tmp = metadata[metadata["Type"].isin(['R_Chemo', 'R_Immuno'])]
tmp

Sample_ID      Type Response (RECIST)   Response  Gender  Age         OS  \
11  T_238402   R_Chemo                PR  Responder    Male   57  10.000000   
12  T_238867   R_Chemo                PR  Responder  Female   74   4.000000   
13  T_239051   R_Chemo                PR  Responder    Male   54   9.000000   
14  T_240194   R_Chemo                PR  Responder    Male   61   5.000000   
15  T_240610   R_Chemo                PR  Responder  Female   63   6.000000   
16  T_242062   R_Chemo                PR  Responder    Male   38   9.700000   
17  T_242127   R_Chemo                PR  Responder    Male   64   5.666667   
18  T_242471   R_Chemo                PR  Responder  Female   55   9.366667   
19  T_250450   R_Chemo                PR  Responder  Female   34  15.633333   
20  T_251611   R_Chemo                PR  Responder    Male   65  14.166667   
21  T_249623  R_Immuno                PR  Responder    Male   66  16.600000   
22  T_251610  R_Immuno                PR  Responder    Male   51  14.166667   
23  T_252216  R_Immuno                PR  Responder    Male   65   9.800000   
24  T_252234  R_Immuno                PR  Responder    Male   68  13.566667   
25  T_259889  R_Immuno                PR  Responder  Female   67   5.633333   
26  T_241815  R_Immuno                PR  Responder    Male   39  13.933333   
27  T_241832  R_Immuno                PR  Responder  Female   46  10.233333   
28  T_183331  R_Immuno                CR  Responder    Male   59  10.000000   
29  T_237916  R_Immuno                CR  Responder    Male   61  10.000000   

    Status        PFS  progress  ...    HLA-DPB1        IDO1       LAG3  \
11       0  10.000000         0  ...   17.043761    0.322369   1.132123   
12       1   4.000000         1  ...    3.726628    0.181735   0.041485   
13       0   6.000000         1  ...   18.166966    0.168021   3.763534   
14       1   5.000000         1  ...   16.696444    0.268206   7.101982   
15       1   6.000000         1  ...    8.447781    1.109735   0.502622   
16       1   7.666667         1  ...   16.925382    0.288234   0.986938   
17       1   5.666667         1  ...    8.389680    0.119481   1.448945   
18       1   5.266667         1  ...   18.143271    0.491979   0.799094   
19       0  15.633333         0  ...   22.989377    0.392304   2.179103   
20       0  14.166667         0  ...   29.143946    0.402033   1.862450   
21       0  16.600000         0  ...   48.735085    2.394222   3.797951   
22       0   7.300000         1  ...   32.313304    3.907125   3.372959   
23       0   9.800000         0  ...   60.390747    1.487132   2.280821   
24       0  13.566667         0  ...    5.377796    0.339578   0.350073   
25       0   5.633333         0  ...   34.795357    6.790712   4.408630   
26       0  13.933333         0  ...  120.939488  350.762473  51.279580   
27       0  10.233333         0  ...  110.462860    4.581624   2.556170   
28       0  10.000000         0  ...    7.173645   13.802998   6.426440   
29       0  10.000000         0  ...  146.495542    4.071501   3.945612   

       TIGIT      ICOS       CD27      CD274  PDCD1LG2     CTLA4     HAVCR2  
11  0.213005  0.295988   0.392561   0.779037  0.497911  0.646686   2.719774  
12  0.000000  0.024102   0.479494   0.401197  0.026065  0.000000   0.157881  
13  0.408921  0.139272   0.720380   0.820308  0.331344  0.136929   1.778970  
14  0.460763  0.047427   0.896346   0.926090  0.461597  0.174860   1.592179  
15  0.312700  0.280337   0.325328   2.692137  5.608471  0.401948   1.377248  
16  0.358819  0.365646   0.892743   2.553730  1.851262  0.674054   6.559424  
17  0.085526  0.316921   1.812645   0.836951  1.649352  0.194743   5.806224  
18  0.119194  0.163120   1.273087   1.397778  3.500865  0.154208   8.209005  
19  0.269583  0.208115   0.345022   1.243557  3.413391  0.085256   3.010499  
20  0.501074  0.219549   1.372714   0.883509  3.154338  0.308365   5.778243  
21  1.185142  1.560742   0.214135   3.100994  1.978811  1.12440

In [92]:
tmp.to_csv("/Volumes/Extreme_Pro/Samsung_T5/东方肝胆外科合作项目/data/RNA-seq/Chemo_Immuno/DEP/R_Chemo_R_Immuno/R_Chemo_R_Immuno_ana.csv", index=False)

In [87]:
expression_matrix

T_240147     T_241235      T_242038     T_242444      T_244938  \
SYMBOL                                                                      
ATAD3B     2.527509     2.331942      2.460850     1.924898      2.370334   
PRDM16     0.137172     0.016004      0.010525     0.801938      0.252774   
PEX10      3.684147     2.424659      5.164162     2.291524      5.312348   
PEX14      5.209140     4.531436     12.936899     3.351609      7.200850   
PLCH2      0.303673     0.101226      0.256771     1.154040      0.362300   
...             ...          ...           ...          ...           ...   
MT-ND6   986.952354  1146.059241   4494.936989  1779.171097   3692.972505   
MT-TE     33.979276    46.061740    114.712148    39.414878     96.236889   
MT-CYB  4981.690357  5844.307036  16650.376150  5322.586570  13655.617420   
MT-TT      5.074827     7.104903      1.557485    12.636649      1.289888   
MT-TP   2844.022605  1873.395817   2999.166195  4390.863831   3610.625164   

           T_248561     T_250599     T_252664      T_253154     T_258375  ...  \
SYMBOL                                                                    ...   
ATAD3B     5.075480     2.731868     5.265348      3.864366     2.663702  ...   
PRDM16     0.017142     0.079891     1.938310      0.370307     0.568922  ...   
PEX10      5.378688     3.038519     2.744660      2.839971     2.762170  ...   
PEX14      8.122877     8.854938     5.583458     15.811962     3.015439  ...   
PLCH2      0.499535     0.242820     0.105764      0.699637     0.077683  ...   
...             ...          ...          ...           ...          ...  ...   
MT-ND6  1244.370042   975.812959   993.477961   5095.237738   749.120425  ...   
MT-TE     44.282830    45.234728    36.450020    112.776500    35.075873  ...   
MT-CYB  2999.205395  5678.632495  6377.902600  20969.178900  2493.111964  ...   
MT-TT      9.512812     5.373960    16.628439      0.000000     1.122558  ...   
MT-TP    660.468947   970.157926  1714.807764   2733.545392   531.333152  ...   

           T_242471      T_249623     T_250450     T_251610     T_251611  \
SYMBOL                                                                     
ATAD3B     5.165169      1.638998     2.765437     3.372115     3.402105   
PRDM16     0.479785      0.115936     0.093400     0.131921     0.152020   
PEX10      3.455357      2.058401     3.473355     3.367540     3.751421   
PEX14      7.090810      2.848242    10.391797     5.172001     8.608873   
PLCH2      1.767715      0.000000     0.177232     1.389179     1.068394   
...             ...           ...          ...          ...          ...   
MT-ND6   931.114288  16581.040610  1875.040102   560.697043  1472.207880   
MT-TE     53.087608    301.548487   149.395503    35.192337    86.074054   
MT-CYB  3242.629640  27321.428760  7946.464667  3078.456786  8471.935537   
MT-TT      6.500080    257.351213     1.382179     3.754285     7.498876   
MT-TP   1953.332803  16071.394020  1269.084077   733.145532  1282.197498   

           T_252216     T_252234     T_259889     T_183331     T_237916  
SYMBOL                                                                   
ATAD3B     2.583519     3.715302     6.402791     3.246631    15.654151  
PRDM16     0.074684     0.194914     1.032753     0.915904     0.712621  
PEX10      5.743271     2.624681     2.656979     2.461080     3.633793  
PEX14      6.744844     2.875431     6.437494     4.224509     8.951129  
PLCH2      1.012265     1.044162     3.920124     0.603116     1.788220  
...             ...          ...          ...          ...          ...  
MT-ND6  2248.985562   534.336824  1455.614041   136.452358   705.094359  
MT-TE     91.620266    13.795077    24.364495     5.248860    12.207439  
MT-CYB  8388.149474  3242.506339  4567.542066  2941.237439  9433.955828  
MT-TT      3.684032    26.097180     0.000000    14.816100    27.539750  
MT-TP   1168.054753  1667.088687  2491.383627   250.856902  2011.903182  

[16

In [93]:
data = expression_matrix[tmp['Sample_ID'].to_list()]
data

T_238402     T_238867      T_239051      T_240194     T_240610  \
SYMBOL                                                                       
ATAD3B      4.647684     3.884499      4.584523     10.410701     4.308672   
PRDM16      1.363787     2.076841      0.052503      0.114938     0.358566   
PEX10       3.567714     3.486233      5.249250      5.145016     2.095672   
PEX14       6.578514     4.107752     14.041770     12.358503     2.996589   
PLCH2       0.136035     0.064509      1.219934      5.475634     1.511998   
...              ...          ...           ...           ...          ...   
MT-ND6  14208.770790  1946.467753   1227.230211   1034.101473  1586.453518   
MT-TE     663.122100    93.705616     14.863752      8.134804    16.027970   
MT-CYB  52112.633960  6061.630640  11424.056810  11736.716300  4668.027911   
MT-TT      82.562633     5.762645     24.419021     18.899040     0.000000   
MT-TP   17054.618520  2102.094191    876.930125    492.514542  2819.037004   

           T_242062     T_242127     T_242471     T_250450     T_251611  \
SYMBOL                                                                    
ATAD3B     4.998995     2.812769     5.165169     2.765437     3.402105   
PRDM16     0.058180     0.208013     0.479785     0.093400     0.152020   
PEX10      2.542701     4.063374     3.455357     3.473355     3.751421   
PEX14      5.680090     7.764326     7.090810    10.391797     8.608873   
PLCH2      0.109187     0.428934     1.767715     0.177232     1.068394   
...             ...          ...          ...          ...          ...   
MT-ND6   460.176425   312.860656   931.114288  1875.040102  1472.207880   
MT-TE     50.046327     6.039864    53.087608   149.395503    86.074054   
MT-CYB  3549.085668  5017.398247  3242.629640  7946.464667  8471.935537   
MT-TT      0.662293     5.525103     6.500080     1.382179     7.498876   
MT-TP    650.527716  2076.092325  1953.332803  1269.084077  1282.197498   

            T_249623     T_251610     T_252216     T_252234     T_259889  \
SYMBOL                                                                     
ATAD3B      1.638998     3.372115     2.583519     3.715302     6.402791   
PRDM16      0.115936     0.131921     0.074684     0.194914     1.032753   
PEX10       2.058401     3.367540     5.743271     2.624681     2.656979   
PEX14       2.848242     5.172001     6.744844     2.875431     6.437494   
PLCH2       0.000000     1.389179     1.012265     1.044162     3.920124   
...              ...          ...          ...          ...          ...   
MT-ND6  16581.040610   560.697043  2248.985562   534.336824  1455.614041   
MT-TE     301.548487    35.192337    91.620266    13.795077    24.364495   
MT-CYB  27321.428760  3078.456786  8388.149474  3242.506339  4567.542066   
MT-TT     257.351213     3.754285     3.684032    26.097180     0.000000   
MT-TP   16071.394020   733.145532  1168.054753  1667.088687  2491.383627   

           T_241815     T_241832     T_183331     T_237916  
SYMBOL                                                      
ATAD3B     3.424163     2.305052     3.246631    15.654151  
PRDM16     0.418049     0.308420     0.915904     0.712621  
PEX10      2.370445     2.159982     2.461080     3.633793  
PEX14      4.041545     4.440458     4.224509     8.951129  
PLCH2      0.800662     0.604611     0.603116     1.788220  
...             ...          ...          ...          ...  
MT-ND6   363.926722   517.275927   136.452358   705.094359  
MT-TE     10.934525    20.718606     5.248860    12.207439  
MT-CYB  2351.625845  1310.421967  2941.237439  9433.955828  
MT-TT      4.034664     1.547169    14.816100    27.539750  
MT-TP   1284.447253   397.190031   250.856902  2011.903182  

[16850 rows x 19 columns]

In [94]:
data.to_csv("/Volumes/Extreme_Pro/Samsung_T5/东方肝胆外科合作项目/data/RNA-seq/Chemo_Immuno/DEP/R_Chemo_R_Immuno/R_Chemo_R_Immuno_matrix.csv")

In [6]:
mut = pd.read_csv("/Volumes/Extreme_Pro/Samsung_T5/东方肝胆外科合作项目/data/WES/Results/Top32_onco_matrix_binary_v4.csv", index_col = 0)
mut

T_238402  T_242062  T_251610  T_238867  T_250450  T_242127  T_252234  \
Symbol                                                                         
TP53           1         1         1         1         1         0         0   
BRCA1          1         1         0         0         0         1         1   
KMT2D          1         0         0         0         0         1         0   
ARID2          0         0         1         0         0         0         0   
MET            1         0         0         0         0         0         0   
RB1            0         1         0         0         0         0         0   
TSC2           1         0         0         0         0         0         0   
ARID1A         0         0         0         0         0         0         0   
KEAP1          1         0         0         1         0         0         0   
FGFR4          0         0         0         0         1         0         0   
BRCA2          0         1         0         0         0         0         0   
NOTCH1         1         0         0         0         0         1         0   
SPTA1          0         1         0         0         0         0         0   
KRAS           0         0         0         0         0         0         0   
PIK3CA         0         0         0         0         0         0         0   
SMAD4          0         0         0         0         0         0         0   
NFE2L2         0         0         0         0         1         0         0   
KDM6A          0         0         0         0         0         0         0   
PBRM1          0         1         0         0         0         0         0   
FGFR3          0         0         0         0         0         0         0   
JAK3           0         1         0         0         0         0         0   
MPL            1         0         0         0         0         0         0   
ZNF536         0         0         0         0         0         0         0   
TERT           0         0         0         0         0         0         0   
IDH1           0         0         0         0         0         0         0   
IDH2           0         0         0         0         0         0         0   
FGFR2          0         0         0         0         0         0         0   
FBXW7          0         0         0         0         0         0         0   
EGFR           0         0         0         0         0         0         0   
FGFR1          0         0         0         0         0         0         0   
CDKN2A         0         0         0         1         0         0         0   

        T_183331  T_240610  T_240392  ...  T_248561  T_241235  T_252664  \
Symbol                                ...                                 
TP53           0         0         0  ...         1         1         1   
BRCA1          0         0         0  ...         0         0         0   
KMT2D          1         1         1  ...         1         1         0   
ARID2          0         0         0  ...         1         0         1   
MET            1         1         0  ...         0         0         0   
RB1            0         0         0  ...         0         0         0   
TSC2           0         0         0  ...         0         0         0   
ARID1A         1         1         0  ...         0         0         0   
KEAP1          1         0         0  ...         0         0         0   
FGFR4          0         0         0  ...         0         0         0   
BRCA2          0         0         0  ...         0         0         1   
NOTCH1         0         0         0  ...         0         0         0   
SPTA1          0         1         0  ...         0         0         0   
KRAS           0         1         0  ...         0         0         0   
PIK3CA         0         0         0  ...         1         0         0   
SMAD4          0         0         0  ...         0         1         0   
NFE2L2         0 

In [24]:
mut_ana = pd.read_csv("/Volumes/Extreme_Pro/Samsung_T5/东方肝胆外科合作项目/data/RNA-seq/Chemo_Immuno/RNA_matrix_ana_v2.csv")
mut_ana

id     Group Sample_ID
0   T_240147        NR  T_240147
1   T_241235        NR  T_241235
2   T_242038        NR  T_242038
3   T_242444        NR  T_242444
4   T_244938        NR  T_244938
5   T_248561        NR  T_248561
6   T_250599        NR  T_250599
7   T_252664        NR  T_252664
8   T_253154        NR  T_253154
9   T_258375        NR  T_258375
10  T_257310        NR  T_257310
11  T_238402   R_Chemo  T_238402
12  T_238867   R_Chemo  T_238867
13  T_239051   R_Chemo  T_239051
14  T_240194   R_Chemo  T_240194
15  T_240610   R_Chemo  T_240610
16  T_242062   R_Chemo  T_242062
17  T_242127   R_Chemo  T_242127
18  T_242471   R_Chemo  T_242471
19  T_250450   R_Chemo  T_250450
20  T_251611   R_Chemo  T_251611
21  T_249623  R_Immuno  T_249623
22  T_251610  R_Immuno  T_251610
23  T_252216  R_Immuno  T_252216
24  T_252234  R_Immuno  T_252234
25  T_259889  R_Immuno  T_259889
26  T_241815  R_Immuno  T_241815
27  T_241832  R_Immuno  T_241832
28  T_183331  R_Immuno  T_183331
29  T_237916  R_Immuno  T_237916

In [8]:
outpath = r"/Volumes/Extreme_Pro/Samsung_T5/东方肝胆外科合作项目/data/RNA-seq/Chemo_Immuno/Mutation/"

In [21]:
cancer_types = mut_ana['Group'].unique()
df = mut_ana.copy()
df['Sample'] = df['id']
df

id     Group Sample_ID    Sample
0   T_238402   R_Chemo  T_238402  T_238402
1   T_238867   R_Chemo  T_238867  T_238867
2   T_239051   R_Chemo  T_239051  T_239051
3   T_240194   R_Chemo  T_240194  T_240194
4   T_240610   R_Chemo  T_240610  T_240610
5   T_242062   R_Chemo  T_242062  T_242062
6   T_242127   R_Chemo  T_242127  T_242127
7   T_242471   R_Chemo  T_242471  T_242471
8   T_250450   R_Chemo  T_250450  T_250450
9   T_251611   R_Chemo  T_251611  T_251611
10  T_249623  R_Immuno  T_249623  T_249623
11  T_251610  R_Immuno  T_251610  T_251610
12  T_252216  R_Immuno  T_252216  T_252216
13  T_252234  R_Immuno  T_252234  T_252234
14  T_259889  R_Immuno  T_259889  T_259889
15  T_241815  R_Immuno  T_241815  T_241815
16  T_241832  R_Immuno  T_241832  T_241832
17  T_183331  R_Immuno  T_183331  T_183331
18  T_237916  R_Immuno  T_237916  T_237916

In [22]:
cancer_types = mut_ana['Group'].unique()
cancer_types

array(['R_Chemo', 'R_Immuno'], dtype=object)

In [23]:
cancer_types = mut_ana['Group'].unique()
df = mut_ana.copy()

for cancer in cancer_types:
    df['Group'] = df['Group'].apply(lambda x: cancer if x == cancer else "The Rest")
    # df['Group'] = df['Type']
    df = df.sort_values("Group")
    if len(df.Group.unique()) == 2:
        dep = calculate_group_differences(mut, df, group1='The Rest', group2=cancer, output_file=f"{outpath}/DEP/{cancer}_DEP.csv", method='wilcox')
        dep.to_csv(f"{outpath}/{cancer}_the_rest_dep.csv")

        overexpression_dep = dep[(dep['FC'] > 1.5) & (dep['p-value'] < 0.05)]
        overexpression_dep.to_csv(f"{outpath}/DEP/{cancer}_the_rest_overexpression_dep.csv")

结果已保存到 /Volumes/Extreme_Pro/Samsung_T5/东方肝胆外科合作项目/data/RNA-seq/Chemo_Immuno/Mutation//DEP/R_Chemo_DEP.csv


In [25]:
merge = pd.merge(mut.T, mut_ana, left_index = True, right_on = "id", how = "inner")
merge

TP53  BRCA1  KMT2D  ARID2  MET  RB1  TSC2  ARID1A  KEAP1  FGFR4  ...  \
11     1      1      1      0    1    0     1       0      1      0  ...   
16     1      1      0      0    0    1     0       0      0      0  ...   
22     1      0      0      1    0    0     0       0      0      0  ...   
12     1      0      0      0    0    0     0       0      1      0  ...   
19     1      0      0      0    0    0     0       0      0      1  ...   
17     0      1      1      0    0    0     0       0      0      0  ...   
24     0      1      0      0    0    0     0       0      0      0  ...   
28     0      0      1      0    1    0     0       1      1      0  ...   
15     0      0      1      0    1    0     0       1      0      0  ...   
18     0      0      0      0    1    1     0       0      0      0  ...   
14     0      0      0      0    0    1     0       0      0      0  ...   
23     0      0      0      0    0    0     1       0      0      0  ...   
27     0      0      0      0    0    0     0       1      0      0  ...   
26     0      0      0      0    0    0     0       0      0      1  ...   
13     0      0      0      0    0    0     0       0      0      0  ...   
20     0      0      0      0    0    0     0       0      0      0  ...   
5      1      0      1      1    0    0     0       0      0      0  ...   
1      1      0      1      0    0    0     0       0      0      0  ...   
7      1      0      0      1    0    0     0       0      0      0  ...   
9      1      0      0      0    0    0     0       0      0      0  ...   
4      0      1      0      1    0    0     1       0      0      1  ...   
2      0      1      0      0    0    1     0       0      0      0  ...   
6      0      1      0      0    0    0     0       0      0      0  ...   
10     0      0      0      0    0    0     0       0      0      0  ...   
0      0      0      0      0    0    0     0       0      0      0  ...   
8      0      0      0      0    0    0     0       0      0      0  ...   

    IDH1  IDH2  FGFR2  FBXW7  EGFR  FGFR1  CDKN2A        id     Group  \
11     0     0      0      0     0      0       0  T_238402   R_Chemo   
16     0     0      0      0     0      0       0  T_242062   R_Chemo   
22     0     0      0      0     0      0       0  T_251610  R_Immuno   
12     0     0      0      0     0      0       1  T_238867   R_Chemo   
19     0     0      0      0     0      0       0  T_250450   R_Chemo   
17     0     0      0      0     0      0       0  T_242127   R_Chemo   
24     0     0      0      0     0      0       0  T_252234  R_Immuno   
28     0     0      0      0     0      0       0  T_183331  R_Immuno   
15     0     0      0      0     0      0       0  T_240610   R_Chemo   
18     0     0      0      0     0      1       0  T_242471   R_Chemo   
14     1     0      0      0     0      0       0  T_240194   R_Chemo   
23     0     0      0      0     0      0       0  T_252216  R_Immuno   
27     0     0      0      0     0      0       0  T_241832  R_Immuno   
26     0     0      0      0     0      0       0  T_241815  R_Immuno   
13     0     0      0      0     0      0       0  T_239051   R_Chemo   
20     0     0      0      0     0      0       0  T_251611   R_Chemo   
5      0     0      0      0     1      0       0  T_248561        NR   
1      0     0      0      0     0      0       0  T_241235        NR   
7      0     0      0      0     0      0       0  T_252664        NR   
9      0     0      0      1     0      0       0  T_258375        NR   
4      0     0      0      0     0      0       0  T_244938        NR   
2      0     0      0      0     0      0       0  T_242038        NR   
6      0     0      0      0     0      0       0  T_250599        NR   
10     0     0      1      0     0      0       0  T_257310        NR   
0      0     0      0      0     0      0       0  T_240147        NR   
8      0     0      0      0     0      0       0  T_

In [26]:
merge.to_csv("/Volumes/Extreme_Pro/Samsung_T5/东方肝胆外科合作项目/data/RNA-seq/Chemo_Immuno/Mutation/Mutation_data.csv")

In [29]:
rna = pd.read_csv("/Volumes/Extreme_Pro/Samsung_T5/东方肝胆外科合作项目/data/RNA-seq/Chemo_Immuno/RNA_matrix_FPKM_tnm.csv", index_col = 0)
rna = rna.T
rna

id           ATAD3B  LOC100996583    PRDM16     PEX10  LOC100129534  \
T_240147   2.527509      0.107009  0.137172  3.684147      2.699547   
T_241235   2.331942      0.582617  0.016004  2.424659      0.194817   
T_242038   2.460850      0.328415  0.010525  5.164162      0.768713   
T_242444   1.924898      2.432893  0.801938  2.291524      0.858711   
T_244938   2.370334      2.175914  0.252774  5.312348      0.459794   
T_248561   5.075480      0.735495  0.017142  5.378688      6.068913   
T_250599   2.731868      0.113317  0.079891  3.038519      1.031479   
T_252664   5.265348      0.949627  1.938310  2.744660      0.379960   
T_253154   3.864366      0.525238  0.370307  2.839971      0.637472   
T_258375   2.663702      0.710117  0.568922  2.762170      0.625872   
T_257310   2.233076      0.386164  1.259183  1.957493      0.577481   
T_238402   4.647684      0.276339  1.363787  3.567714      1.437377   
T_238867   3.884499      0.911345  2.076841  3.486233      0.342359   
T_239051   4.584523      0.234048  0.052503  5.249250      0.913050   
T_240194  10.410701      0.697393  0.114938  5.145016      1.580547   
T_240610   4.308672      1.570365  0.358566  2.095672      1.250763   
T_241815   3.424163      1.488830  0.418049  2.370445      0.331892   
T_241832   2.305052      1.060280  0.308420  2.159982      0.721198   
T_242062   4.998995      0.139653  0.058180  2.542701      1.216726   
T_242127   2.812769      0.748953  0.208013  4.063374      0.519423   
T_242471   5.165169      0.316298  0.479785  3.455357      1.137945   
T_249623   1.638998      0.452215  0.115936  2.058401      0.470438   
T_250450   2.765437      0.437175  0.093400  3.473355      0.265296   
T_251610   3.372115      0.554147  0.131921  3.367540      0.329416   
T_251611   3.402105      0.395308  0.152020  3.751421      1.062367   
T_252216   2.583519      0.906295  0.074684  5.743271      0.673442   
T_252234   3.715302      2.172205  0.194914  2.624681      0.414286   
T_259889   6.402791      0.000000  1.032753  2.656979      0.825432   
T_183331   3.246631      0.404984  0.915904  2.461080      1.143541   
T_237916  15.654151      4.674010  0.712621  3.633793      1.602372   

id        RPL23AP21      PEX14     PLCH2      SPSB1    SLC2A5  ...  \
T_240147   0.024436   5.209140  0.303673   6.562385  0.919162  ...   
T_241235   0.069690   4.531436  0.101226   3.858884  2.003343  ...   
T_242038   0.162491  12.936899  0.256771   9.311820  0.015500  ...   
T_242444   0.008819   3.351609  1.154040  12.379691  1.547355  ...   
T_244938   0.072462   7.200850  0.362300   7.234641  0.500629  ...   
T_248561   0.005090   8.122877  0.499535   5.261379  0.347120  ...   
T_250599   0.198387   8.854938  0.242820   6.174742  0.502715  ...   
T_252664   0.455948   5.583458  0.105764  10.011900  6.433115  ...   
T_253154   0.026654  15.811962  0.699637   2.134429  0.115681  ...   
T_258375   0.048047   3.015439  0.077683  10.034457  3.977024  ...   
T_257310   0.058789   4.390371  0.346650   6.366229  0.113907  ...   
T_238402   0.136725   6.578514  0.136035  11.153131  1.799789  ...   
T_238867   0.023123   4.107752  0.064509   5.798353  0.019116  ...   
T_239051   0.329586  14.041770  1.219934   4.215145  0.276150  ...   
T_240194   0.364009  12.358503  5.475634   4.159016  0.065827  ...   
T_240610   0.000000   2.996589  1.511998  15.337476  0.972747  ...   
T_241815   0.000000   4.041545  0.800662   5.094225  0.321216  ...   
T_241832   0.006208   4.440458  0.604611  27.042352  3.687590  ...   
T_242062   0.047836   5.680090  0.109187  17.438107  1.061147  ...   
T_242127   0.050675   7.764326  0.428934   8.654124  0.282777  ...   
T_242471   0.016051   7.090810  1.767715  15.482633  7.309649  ...   
T_249623   0.000000   2.848242  0.000000   5.805400  2.134246  ...   
T_250450   0.044370  10.391797  0.177232   2.126492  0.055020  ...   
T_251610   0.078336   5.172001  1.389179   8.890642  2.114672  ...   
T_251611   0.150452   8.608873  1.068394   7.545

In [28]:
rna_ana = pd.read_csv("/Volumes/Extreme_Pro/Samsung_T5/东方肝胆外科合作项目/data/RNA-seq/Chemo_Immuno/RNA_matrix_ana_v2.csv")
rna_ana

id     Group Sample_ID       Response
0   T_240147        NR  T_240147  Non_responder
1   T_241235        NR  T_241235  Non_responder
2   T_242038        NR  T_242038  Non_responder
3   T_242444        NR  T_242444  Non_responder
4   T_244938        NR  T_244938  Non_responder
5   T_248561        NR  T_248561  Non_responder
6   T_250599        NR  T_250599  Non_responder
7   T_252664        NR  T_252664  Non_responder
8   T_253154        NR  T_253154  Non_responder
9   T_258375        NR  T_258375  Non_responder
10  T_257310        NR  T_257310  Non_responder
11  T_238402   R_Chemo  T_238402      Responder
12  T_238867   R_Chemo  T_238867      Responder
13  T_239051   R_Chemo  T_239051      Responder
14  T_240194   R_Chemo  T_240194      Responder
15  T_240610   R_Chemo  T_240610      Responder
16  T_242062   R_Chemo  T_242062      Responder
17  T_242127   R_Chemo  T_242127      Responder
18  T_242471   R_Chemo  T_242471      Responder
19  T_250450   R_Chemo  T_250450      Responder
20  T_251611   R_Chemo  T_251611      Responder
21  T_249623  R_Immuno  T_249623      Responder
22  T_251610  R_Immuno  T_251610      Responder
23  T_252216  R_Immuno  T_252216      Responder
24  T_252234  R_Immuno  T_252234      Responder
25  T_259889  R_Immuno  T_259889      Responder
26  T_241815  R_Immuno  T_241815      Responder
27  T_241832  R_Immuno  T_241832      Responder
28  T_183331  R_Immuno  T_183331      Responder
29  T_237916  R_Immuno  T_237916      Responder

In [49]:
gene_list = pd.read_clipboard()
gene_list = gene_list.Gene.to_list()
gene_list

['TAB3',
 'TRADD',
 'TRAF5',
 'CHUK',
 'SOS1',
 'SOS2',
 'MAP2K4',
 'MAPK8',
 'TIA1',
 'ZMYM2',
 'FGFR1OP2',
 'CPSF6',
 'GAB1']

In [50]:
tmp = rna[gene_list]
tmp

id            TAB3      TRADD      TRAF5      CHUK      SOS1      SOS2  \
T_240147  1.301705   7.275242   1.730009  2.387220  2.082736  0.993053   
T_241235  2.139688   8.224206   3.124309  1.743547  2.246315  1.747253   
T_242038  1.892361   6.338885   0.148118  1.995422  1.885640  1.597834   
T_242444  1.126848  11.881418   2.946111  2.264057  3.585206  3.308853   
T_244938  3.844397   6.464282   0.839814  2.873519  2.488072  2.609852   
T_248561  0.852868   4.622885   3.618707  1.115685  4.086299  1.716906   
T_250599  5.586281   6.039214   2.311601  3.596653  5.227038  4.257402   
T_252664  2.928390   9.322409   1.956450  3.040598  4.639643  2.626015   
T_253154  3.733512   4.942830   0.692439  2.730863  3.464391  3.659633   
T_258375  2.424747  12.603853   1.965410  4.221586  5.907245  4.919351   
T_257310  3.779837   9.302978   2.113402  3.466913  5.314411  4.975329   
T_238402  1.442590  10.110227   3.834812  1.252994  2.967103  3.884022   
T_238867  1.735458  12.018549   1.580863  2.268160  4.552663  5.264178   
T_239051  5.953469  10.787970   0.820102  2.695537  3.203905  3.331783   
T_240194  4.818207  12.657052   1.880268  1.879227  4.259482  2.414605   
T_240610  1.401759   6.955534   1.450547  3.942947  2.655271  3.280720   
T_241815  1.627070  14.194666   5.209466  3.413947  4.286805  2.497566   
T_241832  2.112796   9.915265   6.587213  4.134524  6.619521  5.584771   
T_242062  2.269648  12.250442   2.359502  2.773515  3.107678  2.931705   
T_242127  1.348349   4.003529   2.979430  1.916824  1.322727  1.139646   
T_242471  1.568110   9.643469   2.333658  2.184729  3.943359  2.982765   
T_249623  3.095665  16.023755   2.792583  5.597752  5.530291  3.911383   
T_250450  4.334763   5.919262   0.829123  4.004174  5.369999  4.253960   
T_251610  3.836335   7.138151   2.883750  3.051070  5.459490  4.215303   
T_251611  5.249980   6.263442   1.188583  3.620709  4.941222  4.452053   
T_252216  6.707830   7.608805   1.437352  3.686700  4.703832  4.444380   
T_252234  6.315282   6.946058  11.630599  2.679022  9.108554  4.494831   
T_259889  3.507882  21.117238   5.768038  3.813600  5.535778  7.803266   
T_183331  4.271063  11.483481   4.520116  3.247632  4.650537  3.367338   
T_237916  2.671586  21.054263   4.564918  2.305561  5.150429  2.437382   

id          MAP2K4     MAPK8       TIA1      ZMYM2   FGFR1OP2      CPSF6  \
T_240147  3.860389  1.384902   7.158064   2.159466   2.459100   5.259160   
T_241235  3.732698  1.551122   5.835620   2.165522   2.204887   6.323511   
T_242038  5.509715  3.240869   2.007027   1.079420   1.572308   3.958527   
T_242444  3.112319  1.649281   8.611259   5.304250   4.320096   5.464028   
T_244938  4.980609  2.472842   5.437562   2.531849   2.445176   4.801269   
T_248561  2.683383  1.890762   7.393127   2.397413   1.195083   6.676222   
T_250599  4.647078  3.116386  12.712056   4.115117   5.593887   6.106688   
T_252664  3.099810  1.904948  15.185059   4.079877   3.528315   6.561004   
T_253154  4.818624  2.685045  10.118937   2.849045   3.259678   5.677400   
T_258375  4.113883  2.274590  11.587477   3.632446   7.282128   6.957908   
T_257310  4.906540  3.326589  13.780039   3.832081   7.143484   5.979647   
T_238402  3.439163  3.165076  15.722645   3.633035   3.630874   5.071460   
T_238867  4.563482  1.611920  12.213458   9.816977   2.757921   6.378281   
T_239051  4.298672  2.968448  19.703012   3.722565   5.727099   5.250848   
T_240194  3.812581  3.294341  29.489024   4.978172   6.815410   3.904706   
T_240610  6.758568  4.420366   4.928635   2.869451   7.090097   5.075524   
T_241815  5.480613  1.573577   9.424394   2.506185   4.284259   7.211273   
T_241832  5.374846  2.923851  13.932947   5.323860  10.777065   7.522122   
T_242062  2.044304  3.099647   5.133950   1.935522   1.597203   6.988386   
T_242127  3.494822  1.712408   2.428470   2.226363   2.231075   3.973368   
T_242471  2.820844  2.005813   4.010795   4.153524   3.953991   7.571646   
T_249623  4.909012  2.575108  11.

In [51]:
merge = pd.merge(rna_ana, tmp, left_on = "id", right_index = True, how = "left")
merge

id     Group Sample_ID       Response      TAB3      TRADD  \
0   T_240147        NR  T_240147  Non_responder  1.301705   7.275242   
1   T_241235        NR  T_241235  Non_responder  2.139688   8.224206   
2   T_242038        NR  T_242038  Non_responder  1.892361   6.338885   
3   T_242444        NR  T_242444  Non_responder  1.126848  11.881418   
4   T_244938        NR  T_244938  Non_responder  3.844397   6.464282   
5   T_248561        NR  T_248561  Non_responder  0.852868   4.622885   
6   T_250599        NR  T_250599  Non_responder  5.586281   6.039214   
7   T_252664        NR  T_252664  Non_responder  2.928390   9.322409   
8   T_253154        NR  T_253154  Non_responder  3.733512   4.942830   
9   T_258375        NR  T_258375  Non_responder  2.424747  12.603853   
10  T_257310        NR  T_257310  Non_responder  3.779837   9.302978   
11  T_238402   R_Chemo  T_238402      Responder  1.442590  10.110227   
12  T_238867   R_Chemo  T_238867      Responder  1.735458  12.018549   
13  T_239051   R_Chemo  T_239051      Responder  5.953469  10.787970   
14  T_240194   R_Chemo  T_240194      Responder  4.818207  12.657052   
15  T_240610   R_Chemo  T_240610      Responder  1.401759   6.955534   
16  T_242062   R_Chemo  T_242062      Responder  2.269648  12.250442   
17  T_242127   R_Chemo  T_242127      Responder  1.348349   4.003529   
18  T_242471   R_Chemo  T_242471      Responder  1.568110   9.643469   
19  T_250450   R_Chemo  T_250450      Responder  4.334763   5.919262   
20  T_251611   R_Chemo  T_251611      Responder  5.249980   6.263442   
21  T_249623  R_Immuno  T_249623      Responder  3.095665  16.023755   
22  T_251610  R_Immuno  T_251610      Responder  3.836335   7.138151   
23  T_252216  R_Immuno  T_252216      Responder  6.707830   7.608805   
24  T_252234  R_Immuno  T_252234      Responder  6.315282   6.946058   
25  T_259889  R_Immuno  T_259889      Responder  3.507882  21.117238   
26  T_241815  R_Immuno  T_241815      Responder  1.627070  14.194666   
27  T_241832  R_Immuno  T_241832      Responder  2.112796   9.915265   
28  T_183331  R_Immuno  T_183331      Responder  4.271063  11.483481   
29  T_237916  R_Immuno  T_237916      Responder  2.671586  21.054263   

        TRAF5      CHUK      SOS1      SOS2    MAP2K4     MAPK8       TIA1  \
0    1.730009  2.387220  2.082736  0.993053  3.860389  1.384902   7.158064   
1    3.124309  1.743547  2.246315  1.747253  3.732698  1.551122   5.835620   
2    0.148118  1.995422  1.885640  1.597834  5.509715  3.240869   2.007027   
3    2.946111  2.264057  3.585206  3.308853  3.112319  1.649281   8.611259   
4    0.839814  2.873519  2.488072  2.609852  4.980609  2.472842   5.437562   
5    3.618707  1.115685  4.086299  1.716906  2.683383  1.890762   7.393127   
6    2.311601  3.596653  5.227038  4.257402  4.647078  3.116386  12.712056   
7    1.956450  3.040598  4.639643  2.626015  3.099810  1.904948  15.185059   
8    0.692439  2.730863  3.464391  3.659633  4.818624  2.685045  10.118937   
9    1.965410  4.221586  5.907245  4.919351  4.113883  2.274590  11.587477   
10   2.113402  3.466913  5.314411  4.975329  4.906540  3.326589  13.780039   
11   3.834812  1.252994  2.967103  3.884022  3.439163  3.165076  15.722645   
12   1.580863  2.268160  4.552663  5.264178  4.563482  1.611920  12.213458   
13   0.820102  2.695537  3.203905  3.331783  4.298672  2.968448  19.703012   
14   1.880268  1.879227  4.259482  2.414605  3.812581  3.294341  29.489024   
15   1.450547  3.942947  2.655271  3.280720  6.758568  4.420366   4.928635   
16   2.359502  2.773515  3.107678  2.931705  2.044304  3.099647   5.133950   
17   2.979430  1.916824  1.322727  1.139646  3.494822  1.712408   2.428470   
18   2.333658  2.184729  3.943359  2.982765  2.820844  2.005813   4.010795   
19   0.829123  4.004174  5.369999  4.253960  5.576652  4.186827  13.033944   
20   1.188583  3.620709  4.941222  4.452053  5.895070  3.206052  12.821977   
21   2.792583  5.597752  5.530291  3.911383  4.909012  2.5751

In [52]:
merge.to_csv("/Volumes/Extreme_Pro/Samsung_T5/东方肝胆外科合作项目/data/RNA-seq/Chemo_Immuno/GSVA_pathway_v2/Selected_genes_R_Immuno.csv", index = False)